In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir --verbose
# For download the models
!pip install huggingface_hub

Using pip 23.1.2 from /opt/conda/lib/python3.10/site-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.3 MB/s eta 0:00:00 0:00:01
  Running command pip subprocess to install build dependencies
    Using cached setuptools-68.1.2-py3-none-any.whl (805 kB)
                                                0.0/84.3 kB ? eta -:--:--
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 3.1 MB/s eta 0:00:00
                                                0.0/26.1 MB ? eta -:--:--
       ━━                                       1.4/26.1 MB 41.7 MB/s eta 0:00:01
       ━━━━━━━━━╸                               6.3/26.1 MB 91.3 MB/s eta 0:00:01
       ━━━━━━━━━━━━━━━━╸                       11.3/26.1 MB 133.3 MB/s eta 0:00:01
       ━━━━━━━━━━━━━━━━━━━━━━━━━               16.9/26.1 MB 152.7 MB/s eta 0:00:01
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━       22.3/26.1 MB 157.1 MB/s eta 0:00:01
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 26.1/26.1 MB 155.8 

In [5]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [6]:
from huggingface_hub import hf_hub_download

In [7]:
from llama_cpp import Llama


ggml_init_cublas: found 2 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5
  Device 1: Tesla T4, compute capability 7.5


In [8]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [9]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

llama.cpp: loading model from /root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/47d28ef5de4f3de523c421f325a2e4e039035bab/llama-2-13b-chat.ggmlv3.q5_1.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_head_kv  = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 5.0e-06
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 9 (mostly Q5_1)
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =    0.11 MB
llama_model_load_internal: u

In [10]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

32

In [11]:
prompt = "Write a linear regression in python"
prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: {prompt}

ASSISTANT:
'''

In [12]:
response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)


llama_print_timings:        load time = 10647.00 ms
llama_print_timings:      sample time =   265.78 ms /   256 runs   (    1.04 ms per token,   963.19 tokens per second)
llama_print_timings: prompt eval time = 10646.90 ms /    39 tokens (  273.00 ms per token,     3.66 tokens per second)
llama_print_timings:        eval time = 91441.35 ms /   255 runs   (  358.59 ms per token,     2.79 tokens per second)
llama_print_timings:       total time = 103439.31 ms


In [13]:
print(response)

{'id': 'cmpl-4e71b93f-c0dd-4ae4-af11-5b5fbdc60eba', 'object': 'text_completion', 'created': 1692467439, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/47d28ef5de4f3de523c421f325a2e4e039035bab/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': "SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.\n\nUSER: Write a linear regression in python\n\nASSISTANT:\n\nTo write a linear regression in Python, you can use the scikit-learn library. Here is an example of how to do this:\n```\nfrom sklearn.linear_model import LinearRegression\nimport pandas as pd\n\n# Load your dataset into a Pandas DataFrame\ndf = pd.read_csv('your_data.csv')\n\n# Create a linear regression object and fit the data\nreg = LinearRegression().fit(df[['x1', 'x2']], df['y'])\n\n# Print the coefficients of the linear regression\nprint(reg.coef_)\n\n# Predict the value of y for a given set of x values\npredictions = reg.predict(df[['x1', 'x2']])

In [14]:
print(response["choices"][0]["text"])

SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: Write a linear regression in python

ASSISTANT:

To write a linear regression in Python, you can use the scikit-learn library. Here is an example of how to do this:
```
from sklearn.linear_model import LinearRegression
import pandas as pd

# Load your dataset into a Pandas DataFrame
df = pd.read_csv('your_data.csv')

# Create a linear regression object and fit the data
reg = LinearRegression().fit(df[['x1', 'x2']], df['y'])

# Print the coefficients of the linear regression
print(reg.coef_)

# Predict the value of y for a given set of x values
predictions = reg.predict(df[['x1', 'x2']])
```
This code assumes that your data is stored in a CSV file called `your_data.csv`, and that you want to use the first two features (`x1` and `x2`) as predictors, and the third feature (`y`) as the response variable. The `LinearRegression()` object is created with no arguments, which will fit the data using t